In [86]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [87]:
k = np.array([273, 248, 127, 333, 198]) # permeability (mD)
phi = np.array([0.21, 0.17, 0.10, 0.25, 0.13]) # porosity (%)
x = np.array([400, 300, 150, 200, 250]) # length (ft)
n = len(x) # number of blocks
t = 5 # time step (days)
t_end = 135//t # after (135) days
p = np.zeros((t_end, 5)) # pressure (psia)
p[0]= 3000 # initial pressure (psia)
h = 50 # height (ft)
w = 500 # width (ft)
B = 1 # FVF (RB/STB)
mu = 1.5 # viscosity (cp)
cp = 2.5e-5 # compressibility (1/psi)
rw = 6/2 * 0.08333 # interal radius (ft)
re = 0.14 * (x[3]**2 + w**2)**0.5 # external radius (ft)
A = h * w # area (ft2)
Bc = 0.001127 # conversion factor to stb
ac = 5.614583 # rb to stb
t = 5 # time step (days)
qsc = np.zeros(5) # well flow rates (STB/D)
qsc[3] = -400 # well flow rate at block 4 (STB/D)
qw = 0 # west boundary flow rate (STB/D)
qe = 0 # east boundary flow rate (STB/D)
G = ((2*np.pi*Bc*k[3]*h)/(B*mu*np.log(re/rw)))

In [88]:
T = (1/(mu*B)) * (2*Bc) / ((x[:-1]/(A*k[:-1])) + (x[1:]/(A*k[1:])))
T

array([14.04424959, 15.71314387, 21.08469675, 20.16215385])

In [89]:
ma = (A * x * phi * cp) / (ac * B * t)
ma

array([1.87012998, 1.13543606, 0.33395178, 1.11317261, 0.7235622 ])

In [90]:
for t_idx in range(1, t_end):
    c = np.zeros((n, 7))
    for i in range(n):
        if i == 0:
            c[i][:4] = [0, -(qw + T[0] + ma[0]), T[0], -qsc[0] - ma[0] * p[t_idx - 1][0]]
            c[i][4:] = [c[i][2] / c[i][1], c[i][3] / c[i][1], 0]
        elif i == n - 1:
            c[i][:4] = [T[-1], -(qe + T[-1] + ma[-1]), 0, -qsc[-1] - ma[-1] * p[t_idx - 1][-1]]
            c[i][4:] = [0, (c[i][3] - (c[i][0] * c[i - 1][5])) / (c[i][1] - (c[i][0] * c[i - 1][4])), 0]
        else:
            c[i][:4] = [T[i - 1], -(T[i - 1] + T[i] + ma[i]), T[i], -qsc[i] - ma[i] * p[t_idx - 1][i]]
            c[i][4:] = [c[i][2] / (c[i][1] - (c[i][0] * c[i - 1][4])),
                        (c[i][3] - (c[i][0] * c[i - 1][5])) / (c[i][1] - (c[i][0] * c[i - 1][4])), 0]
    c[:, -1][-1] = c[:, -2][-1]
    for i in range(0, n - 1)[::-1]:
        c[i, -1] = c[i, -2] - (c[i, -3] * c[i + 1, -1])
    p[t_idx] = c[:, -1]

pwf = p[:, -2]-(-qsc[3]/G)

q = -G*(p[:, -2]-pwf)

df = pd.DataFrame(p, columns=[f'p{i} (psia)' for i in range(1, n+1)])
df['pwf (psia)'] = pwf
df['qsc (STB/D)'] = q
df.head(10)

,p1 (psia),p2 (psia),p3 (psia),p4 (psia),p5 (psia),pwf (psia),qsc (STB/D)
0,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,2970.946645,-400.0
1,2936.795500,2928.379199,2915.681456,2904.883113,2908.178330,2875.829758,-400.0
2,2861.760780,2851.769168,2837.302905,2825.280673,2828.152569,2796.227318,-400.0
3,2784.834133,2774.590593,2759.858076,2747.652209,2750.441053,2718.598854,-400.0
4,2707.612385,2697.329549,2682.555951,2670.321728,2673.097372,2641.268374,-400.0
5,2630.344816,2620.055879,2605.275915,2593.037299,2595.810890,2563.983944,-400.0
6,2553.070140,2542.780257,2527.999305,2515.760007,2518.533280,2486.706653,-400.0
7,2475.794361,2465.504331,2450.723226,2438.483823,2441.257047,2409.430469,-400.0
8,2398.518411,2388.228359,2373.447230,2361.207811,2363.981027,2332.154456,-400.0
9,2321.242435,2310.952379,2296.171247,2283.931825,2286.705039,2254.878470,-400.0


In [91]:
p2 = df[df['pwf (psia)']<1600]
p2

,p1 (psia),p2 (psia),p3 (psia),p4 (psia),p5 (psia),pwf (psia),qsc (STB/D)
18,1625.758606,1615.468549,1600.687417,1588.447994,1591.221209,1559.394640,-400.0
19,1548.482625,1538.192568,1523.411436,1511.172013,1513.945228,1482.118658,-400.0
20,1471.206644,1460.916587,1446.135455,1433.896032,1436.669246,1404.842677,-400.0
21,1393.930663,1383.640606,1368.859474,1356.620051,1359.393265,1327.566696,-400.0
22,1316.654682,1306.364625,1291.583492,1279.344070,1282.117284,1250.290715,-400.0
23,1239.378701,1229.088644,1214.307511,1202.068089,1204.841303,1173.014734,-400.0
24,1162.102720,1151.812663,1137.031530,1124.792108,1127.565322,1095.738753,-400.0
25,1084.826739,1074.536682,1059.755549,1047.516127,1050.289341,1018.462772,-400.0
26,1007.550758,997.260701,982.479568,970.240145,973.013360,941.186791,-400.0


In [92]:
p = p2.drop(columns=['pwf (psia)', 'qsc (STB/D)']).values

for t_idx in range(1, len(p)):
    c = np.zeros((n, 7))
    for i in range(n):
        if i == 0:
            c[i][:4] = [0, -(qw + T[0] + ma[0]), T[0], -qsc[0] - ma[0] * p[t_idx - 1][0]]
            c[i][4:] = [c[i][2] / c[i][1], c[i][3] / c[i][1], 0]
        elif i == n - 1:
            c[i][:4] = [T[-1], -(qe + T[-1] + ma[-1]), 0, -qsc[-1] - ma[-1] * p[t_idx - 1][-1]]
            c[i][4:] = [0, (c[i][3] - (c[i][0] * c[i - 1][5])) / (c[i][1] - (c[i][0] * c[i - 1][4])), 0]
        elif i == 3:
            c[i][:4] = [T[i - 1], -(T[i - 1] + T[i] + ma[i] + G), T[i], (-G*1500) - (ma[i] * p[t_idx - 1][i])]
            c[i][4:] = [c[i][2] / (c[i][1] - (c[i][0] * c[i - 1][4])),
                        (c[i][3] - (c[i][0] * c[i - 1][5])) / (c[i][1] - (c[i][0] * c[i - 1][4])), 0]
        else:
            c[i][:4] = [T[i - 1], -(T[i - 1] + T[i] + ma[i]), T[i], -qsc[i] - ma[i] * p[t_idx - 1][i]]
            c[i][4:] = [c[i][2] / (c[i][1] - (c[i][0] * c[i - 1][4])),
                        (c[i][3] - (c[i][0] * c[i - 1][5])) / (c[i][1] - (c[i][0] * c[i - 1][4])), 0]
    c[:, -1][-1] = c[:, -2][-1]
    for i in range(0, n - 1)[::-1]:
        c[i, -1] = c[i, -2] - (c[i, -3] * c[i + 1, -1])
    p[t_idx] = c[:, -1]

pwf = p[:, -2]-(-qsc[3]/G)

q = -G*(p[:, -2]-1500)

df2 = pd.DataFrame(p, columns=[f'p{i} (psia)' for i in range(1, n+1)])
df2['pwf (psia)'] = 1500
df2['qsc (STB/D)'] = q
df2.iloc[1:, :]

,p1 (psia),p2 (psia),p3 (psia),p4 (psia),p5 (psia),pwf (psia),qsc (STB/D)
1,1557.584494,1548.506441,1535.553867,1524.869478,1527.168159,1500,-342.397337
2,1524.613655,1520.223264,1514.255426,1509.470622,1510.083733,1500,-130.389375
3,1510.354073,1508.455270,1505.907780,1503.877075,1504.092098,1500,-53.378692
4,1504.339376,1503.538459,1502.467319,1501.614570,1501.700401,1500,-22.229038
5,1501.817043,1501.481170,1501.032309,1500.675072,1500.710594,1500,-9.294244
6,1500.760703,1500.620041,1500.432093,1500.282521,1500.297351,1500,-3.889679
7,1500.318452,1500.259562,1500.180879,1500.118262,1500.124466,1500,-1.628201
8,1500.133312,1500.108659,1500.075719,1500.049506,1500.052103,1500,-0.681592


In [94]:
dfc = pd.concat((df[df['pwf (psia)']>1500], df2.iloc[1:, :]))
dfc.tail()

,p1 (psia),p2 (psia),p3 (psia),p4 (psia),p5 (psia),pwf (psia),qsc (STB/D)
4,1504.339376,1503.538459,1502.467319,1501.614570,1501.700401,1500.0,-22.229038
5,1501.817043,1501.481170,1501.032309,1500.675072,1500.710594,1500.0,-9.294244
6,1500.760703,1500.620041,1500.432093,1500.282521,1500.297351,1500.0,-3.889679
7,1500.318452,1500.259562,1500.180879,1500.118262,1500.124466,1500.0,-1.628201
8,1500.133312,1500.108659,1500.075719,1500.049506,1500.052103,1500.0,-0.681592


In [104]:
exp = dfc.iloc[:, -2:]
exp.iloc[:, 1] = exp.iloc[:, 1] * -1
exp['Cum (STB)'] = exp['qsc (STB/D)'].values.cumsum()
exp['Time (days)'] = np.arange(0, 135, 5)
exp.head()

,pwf (psia),qsc (STB/D),Cum (STB),Time (days)
0,2970.946645,400.0,400.0,0
1,2875.829758,400.0,800.0,5
2,2796.227318,400.0,1200.0,10
3,2718.598854,400.0,1600.0,15
4,2641.268374,400.0,2000.0,20


In [105]:
exp.to_csv('exp.csv', index=False)